In [ ]:
import re
import pandas as pd
import requests

BASE_URL = "https://chilamboli.devmorphix.com"

CATEGORY_TO_LETTER = {
    "Sub Junior": "B", "Junior": "J", "Senior": "S", "Combined": "C", "Special": "P"
}

EVENT_NAME_TO_KEY = {
    "Fancy Dress": "FD", "Coloring": "CL", "Cinematic Dance": "CD", "Unified Wheel Chair Dance": "UW",
    "Kavitha Parayanam": "KP", "Bottle Art": "BA",
    "Folk Dance Boys (നാടോടി നൃത്തം)": "FB", "Folk Dance Girls (നാടോടി നൃത്തം)": "FG",
    "Fashion Show": "FS", "KATHA Prasangam (കഥാപ്രസംഗം)": "KT",
    "Instrumental Music - Fusion (വൃന്ദ വാദ്യം)": "VV", "Folk Music (നാടോടിപ്പാട്ട്)": "FM",
    "Vegetable Carving": "VC", "Mono Act": "MA", "Folk Dance Boys": "FB", "Folk Dance Girls": "FG",
    "KATHA Prasangam": "KT", "Vrunda Vadhyam - Fusion": "VV", "Nadodi Pattu": "NP", "MIME": "MM",
    "BAND Melam": "BM", "Chenda Melam": "CM", "Shikkari Melam": "SM", "Folk Dance": "FD"
}

print(f"Config loaded: BASE_URL={BASE_URL}")
print(f"Event keys: {len(EVENT_NAME_TO_KEY)} mapped")

In [ ]:
_slot_counters = {}

def get_event_key(name):
    t = name.strip()
    if t in EVENT_NAME_TO_KEY:
        return EVENT_NAME_TO_KEY[t]
    norm = re.sub(r"\s*\([^)]*\)\s*", "", t).strip()
    for k, v in EVENT_NAME_TO_KEY.items():
        if re.sub(r"\s*\([^)]*\)\s*", "", k).strip() == norm:
            return v
    return None

def get_category_letter(age_category):
    return CATEGORY_TO_LETTER.get(age_category.strip())

def next_chest_number(event_name, age_category):
    cat = get_category_letter(age_category)
    key = get_event_key(event_name)
    if not cat or not key:
        return None
    sid = f"{cat}|{key}"
    _slot_counters[sid] = _slot_counters.get(sid, 0) + 1
    seq = str(_slot_counters[sid]).zfill(3)
    return f"{cat}{key}{seq}", f"{cat}{key}-{seq}"

def reset_counters():
    _slot_counters.clear()

def _fetch_paginated(path, limit=100, **params):
    page, all_data = 1, []
    while True:
        r = requests.get(f"{BASE_URL}{path}", params={**params, "page": page, "limit": limit}, timeout=30)
        r.raise_for_status()
        j = r.json()
        # Handle both response formats: with/without "success" field
        if "success" in j and not j.get("success"):
            raise RuntimeError(j.get("message", "API error"))
        data = j.get("data", [])
        all_data.extend(data)
        meta = j.get("metadata", {})
        total = meta.get("total", 0)
        if len(all_data) >= total or not data:
            break
        page += 1
    return all_data

print("Helpers defined: get_event_key, get_category_letter, next_chest_number, reset_counters, _fetch_paginated")

In [ ]:
# Generate Excel sheet with all registrations and chest numbers
print("Fetching all registrations...")
all_regs = _fetch_paginated("/api/registrations", limit=100)
print(f"  -> fetched {len(all_regs)} registration(s)")

# Fetch schools for school names
print("Fetching schools...")
schools_data = _fetch_paginated("/api/schools", limit=100)
school_map = {s["id"]: s.get("name", "Unknown") for s in schools_data}
print(f"  -> fetched {len(schools_data)} school(s)")

# Generate chest numbers for all registrations
print("\nGenerating chest numbers...")
reset_counters()
excel_data = []

for reg in all_regs:
    event = reg.get("event", {})
    event_name = event.get("name", "")
    age_category = event.get("ageCategory", "")
    
    gen = next_chest_number(event_name, age_category)
    if not gen:
        print(f"[skip] no mapping: {event_name!r} / {age_category!r}")
        continue
    
    raw, disp = gen
    school_name = school_map.get(reg.get("schoolId", ""), "Unknown")
    
    # Format created date
    created_at = reg.get("createdAt", "")
    if created_at:
        try:
            from datetime import datetime
            if isinstance(created_at, str):
                # Try parsing ISO format
                dt = datetime.fromisoformat(created_at.replace("Z", "+00:00"))
                created_at = dt.strftime("%Y-%m-%d %H:%M:%S")
        except:
            pass
    
    excel_data.append({
        "Chest Number": disp,
        "Registration ID": reg.get("id", ""),
        "Event Name": event_name,
        "Age Category": age_category,
        "Team Name": reg.get("teamName", ""),
        "School ID": reg.get("schoolId", ""),
        "School Name": school_name,
        "Event ID": reg.get("eventId", ""),
        "Created At": created_at,
    })

print(f"  -> generated {len(excel_data)} chest number(s)")

# Create DataFrame and export to Excel
df = pd.DataFrame(excel_data)
df = df.sort_values(["Event Name", "Age Category", "Chest Number"])

output_file = "chest_numbers_all_registrations.xlsx"
try:
    df.to_excel(output_file, index=False, engine="openpyxl")
    print(f"\n✓ Excel file generated: {output_file}")
    print(f"  Total rows: {len(df)}")
    print(f"  Columns: {', '.join(df.columns)}")
except ImportError:
    print("\n⚠️  openpyxl not installed. Installing...")
    import subprocess
    import sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "openpyxl"])
    df.to_excel(output_file, index=False, engine="openpyxl")
    print(f"\n✓ Excel file generated: {output_file}")
    print(f"  Total rows: {len(df)}")
    print(f"  Columns: {', '.join(df.columns)}")